##Challenge Data Engineer
En este notebook se realiza el analisis exploratorio de los datos, se proponen y analizan distintas soluciones

In [3]:
!pip install memory-profiler==0.61.0 pyspark polars==1.9.0 emoji==2.14.0 findspark==2.0.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.1/33.1 MB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.9/586.9 kB 34.6 MB/s eta 0:00:00
  Attempting uninstall: polars
    Found existing installation: polars 1.7.1
    Uninstalling polars-1.7.1:
      Successfully uninstalled polars-1.7.1


In [5]:
import findspark
from pyspark.sql import SparkSession
import zipfile
import findspark
import os
from pyspark.sql.functions import desc, to_date, col, sum, rank, when, col, regexp_replace, lower
from pyspark.sql.window import Window

#carga las extensiones para el profiler
%load_ext memory_profiler
#set path para pyspark
findspark.init()

In [3]:
!echo $SPARK_HOME

/usr/local/lib/python3.10/dist-packages/pyspark


In [4]:
!curl "https://drive.usercontent.google.com/download?id=1ig2ngoXFTxP5Pa8muXo02mDTFexZzsis&confirm=xxx" --create-dirs -o "data/tweets.json.zip"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 57.5M  100 57.5M    0     0  22.5M      0  0:00:02  0:00:02 --:--:-- 22.5M


In [6]:
!unzip -n data/tweets.json.zip

Archive:  data/tweets.json.zip
  inflating: farmers-protest-tweets-2021-2-4.json  


In [7]:
file_path = "farmers-protest-tweets-2021-2-4.json"

In [18]:
#obtener una instancia de pyspark
def getSparkInstance(name_app="test_app") -> SparkSession:
    findspark.init()
    spark = SparkSession.builder \
      .appName(name_app) \
      .config("spark.driver.memory", "4g") \
      .config("spark.executor.extraJavaOptions", "--illegal-access=permit") \
      .getOrCreate()
    spark.sparkContext.setLogLevel("ERROR")
    return spark

In [19]:
#crear dataframe solo con las columnas a utilizar, transformando la columna date
#(en formato datetime) a formato date y ademas limpiando el campo username,
#quitando espacios y transformando a minuscula para su conteo.
try:
  spark = getSparkInstance(name_app="test_app")
  data = spark.read.json(file_path).select('date','user.username') \
  .withColumn("date", to_date("date")) \
  .withColumn("username", when(col("username").isNotNull(), regexp_replace(lower('username'), ' ', '')).otherwise(None) )
except (FileNotFoundError, IOError) as e:
  print(f'Error archivo no existe en el path: {file_path}, error -> {e}')


In [20]:
data.show(10)

+----------+---------------+
|      date|       username|
+----------+---------------+
|2021-02-24|arjunsinghpanam|
|2021-02-24|     prdeepnain|
|2021-02-24| parmarmaninder|
|2021-02-24|  anmoldhaliwal|
|2021-02-24|     kotiapreet|
|2021-02-24|      babli_708|
|2021-02-24|varinde17354019|
|2021-02-24|    bitnamsingh|
|2021-02-24|  anmoldhaliwal|
|2021-02-24|      satthiara|
+----------+---------------+
only showing top 10 rows



In [21]:
#Get schema and datatypes of data
data.printSchema()

root
 |-- date: date (nullable = true)
 |-- username: string (nullable = true)



In [23]:
#calcular y añadir a nueva columna la cantidad de tweets de cada usuario
df_conteo_tweets = data.groupBy('date','username').count().withColumnRenamed("count", "tweets")
df_conteo_tweets.orderBy(desc("tweets")).show(5)

+----------+---------------+------+
|      date|       username|tweets|
+----------+---------------+------+
|2021-02-19|       preetm91|   267|
|2021-02-18|neetuanjle_nitu|   195|
|2021-02-17| raajvinderkaur|   185|
|2021-02-13|maandee08215437|   178|
|2021-02-12|ranbirs00614606|   176|
+----------+---------------+------+
only showing top 5 rows



In [24]:
# Se define una ventana para obtener el usuario con más tweets por fecha
window = Window.partitionBy('date').orderBy(col('tweets').desc())
top_twitters_df = df_conteo_tweets.withColumn('top_twitters', rank().over(window)).filter(col('top_twitters') == 1)
top_twitters_df.orderBy(col('date').asc(), col('top_twitters').asc()).show()

+----------+---------------+------+------------+
|      date|       username|tweets|top_twitters|
+----------+---------------+------+------------+
|2021-02-12|ranbirs00614606|   176|           1|
|2021-02-13|maandee08215437|   178|           1|
|2021-02-14|  rebelpacifist|   119|           1|
|2021-02-15|         jot__b|   134|           1|
|2021-02-16|         jot__b|   133|           1|
|2021-02-17| raajvinderkaur|   185|           1|
|2021-02-18|neetuanjle_nitu|   195|           1|
|2021-02-19|       preetm91|   267|           1|
|2021-02-20|mangalj23056160|   108|           1|
|2021-02-21|     surrypuria|   161|           1|
|2021-02-22| preetysaini321|   110|           1|
|2021-02-23|     surrypuria|   135|           1|
|2021-02-24| preetysaini321|   107|           1|
+----------+---------------+------+------------+



In [25]:
max_tweets_data = df_conteo_tweets.groupBy("date").agg(sum("tweets").alias("sum_tweets"))
max_tweets_data.orderBy(col('sum_tweets').desc()).show(100)

+----------+----------+
|      date|sum_tweets|
+----------+----------+
|2021-02-12|     12347|
|2021-02-13|     11296|
|2021-02-17|     11087|
|2021-02-16|     10443|
|2021-02-14|     10249|
|2021-02-18|      9625|
|2021-02-15|      9197|
|2021-02-20|      8502|
|2021-02-23|      8417|
|2021-02-19|      8204|
|2021-02-21|      7532|
|2021-02-22|      7071|
|2021-02-24|      3437|
+----------+----------+



In [26]:
#join de ambas dataframes para setear el resultado solicitado
df_final = top_twitters_df.join(max_tweets_data, "date", "inner").select('*').orderBy(col('sum_tweets').desc())
df_final.show(100)

+----------+---------------+------+------------+----------+
|      date|       username|tweets|top_twitters|sum_tweets|
+----------+---------------+------+------------+----------+
|2021-02-12|ranbirs00614606|   176|           1|     12347|
|2021-02-13|maandee08215437|   178|           1|     11296|
|2021-02-17| raajvinderkaur|   185|           1|     11087|
|2021-02-16|         jot__b|   133|           1|     10443|
|2021-02-14|  rebelpacifist|   119|           1|     10249|
|2021-02-18|neetuanjle_nitu|   195|           1|      9625|
|2021-02-15|         jot__b|   134|           1|      9197|
|2021-02-20|mangalj23056160|   108|           1|      8502|
|2021-02-23|     surrypuria|   135|           1|      8417|
|2021-02-19|       preetm91|   267|           1|      8204|
|2021-02-21|     surrypuria|   161|           1|      7532|
|2021-02-22| preetysaini321|   110|           1|      7071|
|2021-02-24| preetysaini321|   107|           1|      3437|
+----------+---------------+------+-----

In [27]:
#al estar ordenados por la cantidad de tweets por fecha al seleccionar los primeros 10 obtenemos efectivamente el top 10 de tweets en esa fecha
result_list = [(row['date'], row['username']) for row in df_final.limit(10).collect()]
result_list

[(datetime.date(2021, 2, 12), 'ranbirs00614606'),
 (datetime.date(2021, 2, 13), 'maandee08215437'),
 (datetime.date(2021, 2, 17), 'raajvinderkaur'),
 (datetime.date(2021, 2, 16), 'jot__b'),
 (datetime.date(2021, 2, 14), 'rebelpacifist'),
 (datetime.date(2021, 2, 18), 'neetuanjle_nitu'),
 (datetime.date(2021, 2, 15), 'jot__b'),
 (datetime.date(2021, 2, 20), 'mangalj23056160'),
 (datetime.date(2021, 2, 23), 'surrypuria'),
 (datetime.date(2021, 2, 19), 'preetm91')]

In [50]:
import json
from datetime import datetime
from typing import List, Tuple
from collections import Counter

def q1_memory_json(file_path):
  date_counts = Counter()
  date_user_counts = {}

  with open(file_path, 'r') as file:
      for line in file:
          tweet = json.loads(line)
          date = datetime.strptime(tweet['date'][0:10], "%Y-%m-%d").date()
          user = tweet['user']['username']
          date_counts[date] += 1

          if date not in date_user_counts:
              date_user_counts[date] = Counter()
          date_user_counts[date][user] += 1

  # Top 10 fechas con mas tweets
  top_10_dates = date_counts.most_common(10)

  result = []
  for date, _ in top_10_dates:
      # obtiene el usuario con mas tweets por fecha
      top_user = date_user_counts[date].most_common(1)[0][0]
      result.append((date, top_user))

  return result

In [49]:
q1_memory_json(file_path)

[(datetime.date(2021, 2, 12), 'RanbirS00614606'),
 (datetime.date(2021, 2, 13), 'MaanDee08215437'),
 (datetime.date(2021, 2, 17), 'RaaJVinderkaur'),
 (datetime.date(2021, 2, 16), 'jot__b'),
 (datetime.date(2021, 2, 14), 'rebelpacifist'),
 (datetime.date(2021, 2, 18), 'neetuanjle_nitu'),
 (datetime.date(2021, 2, 15), 'jot__b'),
 (datetime.date(2021, 2, 20), 'MangalJ23056160'),
 (datetime.date(2021, 2, 23), 'Surrypuria'),
 (datetime.date(2021, 2, 19), 'Preetm91')]

In [58]:
from collections import Counter, defaultdict
from datetime import datetime
import json
from tqdm.contrib.concurrent import process_map


# Función para procesar cada línea
def process_line(line):
    date_counts = Counter()
    date_user_counts = {}
    tweet = json.loads(line)
    date = datetime.strptime(tweet['date'][0:10], "%Y-%m-%d").date()
    user = tweet['user']['username']
    return date, user

def file_line_iterator(file_path):
    with open(file_path, 'r') as file:
        for line in file:
            yield line

def file_all_lines(file_path):
    # Leer todas las líneas del archivo
    with open(file_path, 'r') as file:
        return file.readlines()

def q1_mem_json_tqdm(file_path):
    #conteo de tweets por fecha
    date_counts = Counter()
    #conteo tweets por fecha y usuario
    date_user_counts = defaultdict(Counter)

    #lines = file_all_lines(file_path)
    lines = file_line_iterator(file_path)

    # Procesar las líneas en paralelo
    results = process_map(process_line, lines, chunksize=5000, desc="Procesando Lineas")



    #para cada usuario y fecha
    for date, user in results:
        date_counts[date] += 1
        date_user_counts[date][user] += 1

    #las 10 fechas con mas tweets
    top_10_dates = date_counts.most_common(10)

    resultado = [(date, date_user_counts[date].most_common(1)[0][0])
                                                    for date, _ in top_10_dates]
    #retorna el resultado ordenado por el campo date
    return resultado


In [59]:
q1_mem_json_tqdm(file_path)

Procesando Lineas: 0it [00:00, ?it/s]

[(datetime.date(2021, 2, 12), 'RanbirS00614606'),
 (datetime.date(2021, 2, 13), 'MaanDee08215437'),
 (datetime.date(2021, 2, 17), 'RaaJVinderkaur'),
 (datetime.date(2021, 2, 16), 'jot__b'),
 (datetime.date(2021, 2, 14), 'rebelpacifist'),
 (datetime.date(2021, 2, 18), 'neetuanjle_nitu'),
 (datetime.date(2021, 2, 15), 'jot__b'),
 (datetime.date(2021, 2, 20), 'MangalJ23056160'),
 (datetime.date(2021, 2, 23), 'Surrypuria'),
 (datetime.date(2021, 2, 19), 'Preetm91')]

In [66]:
import polars as pl
from typing import List, Tuple
from datetime import date
from memory_profiler import profile


def q1_memory_polars(file_path: str) -> List[Tuple[date, str]]:
    # Validar el path
    if not file_path:
        raise ValueError("El path no debe ser nulo o vacío")

    # Cargar el archivo usando Polars con ejecución lazy
    try:
        data = (
            pl.read_ndjson(file_path, ignore_errors=True)
            .lazy()
            .select([
                pl.col('date').str.slice(0, 10).str.strptime(pl.Date).alias('date'),
                pl.col('user').struct.field('username')
                    .str.to_lowercase()
                    .str.replace_all(' ', '')
                    .alias('username')
            ])
            .filter(pl.col('username').is_not_null())
        )
    except (FileNotFoundError, IOError) as e:
        print(f"Error al abrir el archivo en {file_path}: {e}")
        raise

    # Agrupar y contar tweets por fecha y usuario
    df_conteo_tweets = (
        data.group_by(['date', 'username'])
        .agg(pl.len().alias('tweets'))
    )

    # Obtener la suma total de tweets por fecha
    sum_tweets_data = (
        df_conteo_tweets.group_by('date')
        .agg(pl.col('tweets').sum().alias('sum_tweets'))
    )

    # Obtener el usuario con más tweets por fecha
    top_twitters_df = (
        df_conteo_tweets.sort(by=['date', 'tweets'], descending=[False, True])
        .group_by('date')
        .first()
    )

    # Unir ambos DataFrames
    df_final = (
        top_twitters_df.join(sum_tweets_data, on='date')
        .sort('sum_tweets', descending=True)
    )

    # Materializar el DataFrame y obtener los resultados como lista de tuplas
    resultado = df_final.select(['date', 'username']).limit(10).collect().to_numpy().tolist()

    return resultado


In [67]:
q1_memory_polars(file_path)

[[datetime.date(2021, 2, 12), 'ranbirs00614606'],
 [datetime.date(2021, 2, 13), 'maandee08215437'],
 [datetime.date(2021, 2, 17), 'raajvinderkaur'],
 [datetime.date(2021, 2, 16), 'jot__b'],
 [datetime.date(2021, 2, 14), 'rebelpacifist'],
 [datetime.date(2021, 2, 18), 'neetuanjle_nitu'],
 [datetime.date(2021, 2, 15), 'jot__b'],
 [datetime.date(2021, 2, 20), 'mangalj23056160'],
 [datetime.date(2021, 2, 23), 'surrypuria'],
 [datetime.date(2021, 2, 19), 'preetm91']]

###Mediciones de memoria de las implementaciones:

- **PySpark** -> Uso de Memoria: 0.0039 MiB
- **Polars** -> Uso de Memoria: 590 MiB
- **Json** -> Uso de Memoria: 0.00 MiB (consumo de memoria despreciable)
- **Json_Tqdm_full_load** -> Uso de Memoria: 28.13 MiB
- **Json_Tqdm_iterator_load** -> Uso de Memoria: 0.586 MiB


###Mediciones de tiempo de las implementaciones:

- **PySpark** -> Tiempo: 15.20 s
- **Polars** -> Tiempo: 22.74 s
- **Json_Counter** -> Tiempo: 7.79 s
- **Json_Tqdm_full_load** -> Tiempo: 9.10 s
- **Json_Tqdm_iterator_load** -> Tiempo: 8.46 s

##Implementacion con la libreria json y usando estructuras de datos optimizadas como Counter y defaultdict, propias del entorno de Python.

In [86]:
import q1_memory_json as q1
from memory_profiler import memory_usage
import imp
imp.reload(q1)

mem_usage_init = memory_usage()[0]
q1.q1_memory_json(file_path)
mem_usage_end = memory_usage()[0]
print(f"Uso de Memoria Ram: {mem_usage_end - mem_usage_init} MiB")


Uso de Memoria Ram: 0.0 MiB


In [87]:
import q1_memory_json as q1
import time
import imp
imp.reload(q1)

time_init = time.time()
q1.q1_memory_json(file_path)
time_end = time.time()
print(f"Tiempo de ejecución: {time_end - time_init:.2f} segundos")

Tiempo de ejecución: 7.79 segundos


##Implementacion con la libreria polars, seleccionada ya que posee mejor rendimiento que pandas.

In [89]:
import q1_memory_polars as q1
from memory_profiler import memory_usage
import imp
imp.reload(q1)

mem_usage_init = memory_usage()[0]
q1.q1_memory_polars(file_path)
mem_usage_end = memory_usage()[0]
print(f"Uso de Memoria Ram: {mem_usage_end - mem_usage_init} MiB")

Uso de Memoria Ram: 589.5078125 MiB


In [90]:
import q1_memory_polars as q1
import time
import imp
imp.reload(q1)

time_init = time.time()
q1.q1_memory_polars(file_path)
time_end = time.time()
print(f"Tiempo de ejecución: {time_end - time_init:.2f} segundos")

Tiempo de ejecución: 22.74 segundos


##Implementacion con Spark mediante pyspark para procesar datos de forma distribuida.

In [91]:
import q1_memory_pyspark as q1
from memory_profiler import memory_usage
import imp
imp.reload(q1)

mem_usage_init = memory_usage()[0]
q1.q1_memory_pyspark(file_path)
mem_usage_end = memory_usage()[0]
print(f"Uso de Memoria Ram: {mem_usage_end - mem_usage_init} MiB")

Uso de Memoria Ram: 0.00390625 MiB


In [92]:
import q1_memory_pyspark as q1
import time
import imp
imp.reload(q1)

time_init = time.time()
q1.q1_memory_pyspark(file_path)
time_end = time.time()
print(f"Tiempo de ejecución: {time_end - time_init:.2f} segundos")

Tiempo de ejecución: 15.20 segundos


##Implementacion con la libreria TQDM para procesos concurrentes, se añaden dos formas de carga de datos, la primera es leer el archivo completo y la otra es leerlo mediante un iterador.

In [96]:
import q1_memory_tqdm as q1
imp.reload(q1)

mem_usage_init = memory_usage()[0]
q1.q1_memory_tqdm(file_path, load_file='full')
mem_usage_end = memory_usage()[0]
print(f"Uso de Memoria Ram: {mem_usage_end - mem_usage_init} MiB")

Procesando Lineas:   0%|          | 0/117407 [00:00<?, ?it/s]

Uso de Memoria Ram: 28.1328125 MiB


In [99]:
import q1_memory_tqdm as q1
imp.reload(q1)

mem_usage_init = memory_usage()[0]
q1.q1_memory_tqdm(file_path, load_file='iterator')
mem_usage_end = memory_usage()[0]
print(f"Uso de Memoria Ram: {mem_usage_end - mem_usage_init} MiB")

Procesando Lineas: 0it [00:00, ?it/s]

Uso de Memoria Ram: 0.5859375 MiB


In [104]:
import q1_memory_tqdm as q1
imp.reload(q1)

time_init = time.time()
q1.q1_memory_tqdm(file_path, load_file='full')
time_end = time.time()
print(f"Tiempo de ejecución: {time_end - time_init:.2f} segundos")

Procesando Lineas:   0%|          | 0/117407 [00:00<?, ?it/s]

Tiempo de ejecución: 9.10 segundos


In [106]:
import q1_memory_tqdm as q1
imp.reload(q1)

time_init = time.time()
q1.q1_memory_tqdm(file_path, load_file='iterator')
time_end = time.time()
print(f"Tiempo de ejecución: {time_end - time_init:.2f} segundos")

Procesando Lineas: 0it [00:00, ?it/s]

Tiempo de ejecución: 8.46 segundos


##Implementacion Optimizada en Memoria
- Usa un generador para procesar las líneas del archivo, evitando cargar todo en memoria.
- Es más lento debido a la lectura línea por línea, pero ideal para archivos grandes.

In [30]:
import json
import logging
from datetime import datetime
from typing import List, Tuple
from collections import Counter, defaultdict

# Configuración del logger
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

def q1_memory(file_path: str) -> List[Tuple[datetime.date, str]]:
    """
    Procesa un archivo NDJSON línea por línea, contando la cantidad de tweets por fecha
    y por usuario, optimizando el uso de memoria con generadores.

    Args:
        file_path (str): Ruta del archivo NDJSON.

    Returns:
        List[Tuple[datetime.date, str]]: Lista de las 10 fechas con más tweets
        y el usuario más activo en cada una.
    """
    if not file_path:
        logger.error("El path no debe ser nulo o vacío.")
        raise ValueError("El path no debe ser nulo o vacío.")

    date_counts = Counter()
    date_user_counts = defaultdict(Counter)

    # Generador que procesa las líneas del archivo para minimizar el uso de memoria
    def tweet_generator(file):
        try:
            for line in file:
                yield json.loads(line)
        except json.JSONDecodeError as e:
            logger.error(f"Error al decodificar JSON: {e}")
            raise

    try:
        with open(file_path, 'r') as file:
            for tweet in tweet_generator(file):
                # Extracción de datos
                date = datetime.strptime(tweet['date'][0:10], "%Y-%m-%d").date()
                user = tweet['user']['username']

                # Actualización de contadores
                date_counts[date] += 1
                date_user_counts[date][user] += 1

        # Obtener las 10 fechas con más tweets
        top_10_dates = date_counts.most_common(10)

        # Construcción del resultado con list comprehension
        result = [
            (date, date_user_counts[date].most_common(1)[0][0])
            for date, _ in top_10_dates
        ]

        return result

    except (FileNotFoundError, IOError) as e:
        logger.error(f"Error al abrir el archivo: {e}")
        raise


In [28]:
%memit q1_memory(file_path)

peak memory: 268.64 MiB, increment: 0.01 MiB


In [31]:
%time q1_memory(file_path)

CPU times: user 8.43 s, sys: 305 ms, total: 8.74 s
Wall time: 8.73 s


[(datetime.date(2021, 2, 12), 'RanbirS00614606'),
 (datetime.date(2021, 2, 13), 'MaanDee08215437'),
 (datetime.date(2021, 2, 17), 'RaaJVinderkaur'),
 (datetime.date(2021, 2, 16), 'jot__b'),
 (datetime.date(2021, 2, 14), 'rebelpacifist'),
 (datetime.date(2021, 2, 18), 'neetuanjle_nitu'),
 (datetime.date(2021, 2, 15), 'jot__b'),
 (datetime.date(2021, 2, 20), 'MangalJ23056160'),
 (datetime.date(2021, 2, 23), 'Surrypuria'),
 (datetime.date(2021, 2, 19), 'Preetm91')]

##Optimización en tiempo (uso de max y defaultdict):

- Evita la sobrecarga del método most_common(1) usando max en su lugar, lo que se traduce en la reduccion del tiempo de ejecución.
- Usa defaultdict(lambda: defaultdict(int)) para evitar la creación repetida de objetos Counter.

In [33]:
import json
import logging
from datetime import datetime
from typing import List, Tuple
from collections import defaultdict, Counter

# Configuración del logger
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

def q1_time(file_path: str) -> List[Tuple[datetime.date, str]]:
    """
    Procesa un archivo NDJSON línea por línea, contando la cantidad de tweets por fecha
    y usuario, optimizando el tiempo de ejecución.

    Args:
        file_path (str): Ruta del archivo NDJSON.

    Returns:
        List[Tuple[datetime.date, str]]: Lista de las 10 fechas con más tweets
        y el usuario más activo en cada una.
    """
    if not file_path:
        logger.error("El path no debe ser nulo o vacío.")
        raise ValueError("El path no debe ser nulo o vacío.")

    # Contadores optimizados con defaultdict
    date_counts = Counter()
    date_user_counts = defaultdict(lambda: defaultdict(int))

    try:
        with open(file_path, 'r') as file:
            for line in file:
                try:
                    tweet = json.loads(line)
                except json.JSONDecodeError as e:
                    logger.warning(f"Error al decodificar JSON en una línea: {e}")
                    continue  # Ignorar líneas mal formadas y continuar

                # Extracción de datos
                date = datetime.strptime(tweet['date'][0:10], "%Y-%m-%d").date()
                user = tweet['user']['username']

                # Actualización de contadores
                date_counts[date] += 1
                date_user_counts[date][user] += 1

        # Obtener las 10 fechas con más tweets de manera eficiente
        top_10_dates = date_counts.most_common(10)

        # Construcción del resultado con un bucle optimizado
        result = []
        for date, _ in top_10_dates:
            # Uso de max para encontrar el usuario con más tweets en la fecha
            top_user = max(date_user_counts[date].items(), key=lambda x: x[1])[0]
            result.append((date, top_user))

        return result

    except (FileNotFoundError, IOError) as e:
        logger.error(f"Error al abrir el archivo: {e}")
        raise


In [34]:
%memit q1_time(file_path)

peak memory: 270.67 MiB, increment: 0.26 MiB


In [35]:
%time q1_time(file_path)

CPU times: user 8.14 s, sys: 323 ms, total: 8.46 s
Wall time: 8.42 s


[(datetime.date(2021, 2, 12), 'RanbirS00614606'),
 (datetime.date(2021, 2, 13), 'MaanDee08215437'),
 (datetime.date(2021, 2, 17), 'RaaJVinderkaur'),
 (datetime.date(2021, 2, 16), 'jot__b'),
 (datetime.date(2021, 2, 14), 'rebelpacifist'),
 (datetime.date(2021, 2, 18), 'neetuanjle_nitu'),
 (datetime.date(2021, 2, 15), 'jot__b'),
 (datetime.date(2021, 2, 20), 'MangalJ23056160'),
 (datetime.date(2021, 2, 23), 'Surrypuria'),
 (datetime.date(2021, 2, 19), 'Preetm91')]

##Q2 Optimizacion en Memoria

- Procesamiento línea por línea:
Evita cargar todo el archivo en memoria a la vez, lo que es útil para archivos muy grandes.
- Uso eficiente del Counter:
El Counter se actualiza de forma incremental sin mantener listas intermedias en memoria.

- Manejo de errores sin interrupciones:
Si una línea tiene un error de formato, se registra y el proceso continúa, evitando cuellos de botella.

In [85]:
import json
import logging
from collections import Counter
from typing import List, Tuple
import emoji  # Asegúrate de tener instalada la librería emoji: pip install emoji

# Configuración del logger
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

def q2_memory(file_path: str) -> List[Tuple[str, int]]:
    """
    Procesa un archivo NDJSON en streaming para contar las ocurrencias de emojis
    en los tweets, optimizando el uso de memoria.

    Args:
        file_path (str): Ruta del archivo NDJSON.

    Returns:
        List[Tuple[str, int]]: Lista de los 10 emojis más comunes con sus frecuencias.
    """
    # Inicializar un contador para los emojis
    emojis = Counter()

    try:
        # Procesar el archivo línea por línea para minimizar el uso de memoria
        with open(file_path, 'r') as file:
            for line in file:
                try:
                    # Decodificar cada línea del archivo como JSON
                    tweet = json.loads(line)
                except json.JSONDecodeError as e:
                    # Registrar el error y continuar con la siguiente línea
                    logger.warning(f"Error al decodificar JSON: {e}")
                    continue

                # Extraer el contenido del tweet y buscar emojis
                content = tweet.get('content', '')
                emojis_ = emoji.emoji_list(content)

                # Actualizar el contador solo si se encuentran emojis
                if emojis_:
                    emojis.update(emo['emoji'] for emo in emojis_)

    except (FileNotFoundError, IOError) as e:
        # Registrar error si el archivo no se encuentra o no se puede abrir
        logger.error(f"Error al abrir el archivo: {e}")
        raise

    # Devolver los 10 emojis más comunes
    return emojis.most_common(10)

##Q2 Optimización del Tiempo de Ejecución

- Uso de Generadores para Evitar Acumulación de Datos:
En lugar de almacenar todas las líneas o tweets en memoria, usamos generadores.
Esto reduce la sobrecarga de acceso a memoria y acelera la ejecución.

- Actualización Directa del Counter:
Actualizamos el Counter solo si hay emojis presentes, evitando operaciones innecesarias:

- Acceso Mínimo a Estructuras:
Evitamos múltiples accesos a los diccionarios JSON y objetos innecesarios, mejorando la velocidad.

- Manejo Eficiente de Errores:
Si se encuentra un error en una línea JSON, lo reportamos y continuamos la ejecución sin interrupciones.

In [30]:
import json
import logging
from collections import Counter
from typing import List, Tuple
import emoji

# Configuración del logger
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

def q2_time(file_path: str) -> List[Tuple[str, int]]:
    """
    Procesa un archivo NDJSON para contar las ocurrencias de emojis en los tweets,
    optimizando el tiempo de ejecución.

    Args:
        file_path (str): Ruta del archivo NDJSON.

    Returns:
        List[Tuple[str, int]]: Lista de los 10 emojis más comunes con sus frecuencias.
    """
    # Inicializar el contador de emojis
    emojis = Counter()

    try:
        # Abrir el archivo en modo lectura
        with open(file_path, 'r') as file:
            # Generador para extraer el contenido de cada tweet
            tweets = (json.loads(line).get('content', '') for line in file)

            # Extraer emojis y actualizar el contador en un solo paso
            for content in tweets:
                try:
                    emojis_ = emoji.emoji_list(content)
                    if emojis_:
                        emojis.update(emo['emoji'] for emo in emojis_)
                except Exception as e:
                    # Registrar cualquier error inesperado
                    logger.warning(f"Error al procesar emojis: {e}")

    except (FileNotFoundError, IOError) as e:
        # Registrar error si no se puede abrir el archivo
        logger.error(f"Error al abrir el archivo: {e}")
        raise

    # Obtener los 10 emojis más comunes
    return emojis.most_common(10)


In [31]:
%time q2_time(file_path)

CPU times: user 26.3 s, sys: 457 ms, total: 26.7 s
Wall time: 27.5 s


[('🙏', 5049),
 ('😂', 3072),
 ('🚜', 2972),
 ('🌾', 2182),
 ('🇮🇳', 2086),
 ('🤣', 1668),
 ('✊', 1651),
 ('❤️', 1382),
 ('🙏🏻', 1317),
 ('💚', 1040)]

In [32]:
%time q2_memory(file_path)

NameError: name 'q2_memory' is not defined

##Q3 Optimizada en Memoria

- Procesar el archivo línea por línea.
- Actualizar el contador de menciones de forma incremental.
- Usar logger para registrar errores y el progreso del programa.

In [19]:
import re
import json
import logging
from collections import Counter
from typing import List, Tuple

# Configuración del logger
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

def q3_memory(file_path: str) -> List[Tuple[str, int]]:
    """
    Procesa un archivo NDJSON para contar las menciones de usuarios en los tweets,
    optimizando el uso de memoria.

    Args:
        file_path (str): Ruta del archivo NDJSON.

    Returns:
        List[Tuple[str, int]]: Lista de los 10 usuarios más mencionados con sus frecuencias.
    """
    # Compilar la expresión regular para encontrar menciones (@usuario)
    username_pattern = re.compile(r'@(\w+)')

    # Inicializar el contador para los nombres de usuario
    mention_counter = Counter()

    try:
        # Abrir el archivo y procesar línea por línea (minimiza uso de memoria)
        with open(file_path, 'r') as tweet_file:
            for tweet_json in tweet_file:
                try:
                    # Parsear la línea como JSON
                    tweet_data = json.loads(tweet_json.strip())
                except json.JSONDecodeError as e:
                    # Registrar advertencia y continuar con la siguiente línea
                    logger.warning(f"Error al decodificar JSON: {e}")
                    continue

                # Extraer el contenido del tweet si está disponible
                tweet_content = tweet_data.get('content', '')

                # Encontrar todas las menciones de usuario en el contenido
                mentioned_users = username_pattern.findall(tweet_content)

                # Actualizar el contador solo si hay menciones
                if mentioned_users:
                    mention_counter.update(mentioned_users)

    except (FileNotFoundError, IOError) as e:
        # Registrar error si el archivo no se encuentra o no se puede abrir
        logger.error(f"Error al abrir el archivo: {e}")
        raise

    # Obtener los 10 usuarios más mencionados
    top_mentioned_users = mention_counter.most_common(10)

    # Registrar los usuarios más mencionados
    logger.info(f"Top 10 usuarios mencionados: {top_mentioned_users}")

    return top_mentioned_users


In [20]:
q3_memory(file_path)

[('narendramodi', 2261),
 ('Kisanektamorcha', 1836),
 ('RakeshTikaitBKU', 1639),
 ('PMOIndia', 1422),
 ('RahulGandhi', 1125),
 ('GretaThunberg', 1046),
 ('RaviSinghKA', 1015),
 ('rihanna', 972),
 ('UNHumanRights', 962),
 ('meenaharris', 925)]

##Q3 Optimizacion en tiempo de ejecucion

Reducir los accesos a estructuras y trabajar con expresiones generadoras.
Evitar operaciones innecesarias y actualizar el Counter solo si es necesario.
Usar precompilación de la expresión regular para mayor eficiencia.

In [24]:
import re
import json
import logging
from collections import Counter
from typing import List, Tuple

# Configuración del logger
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

def q3_time(file_path: str) -> List[Tuple[str, int]]:
    """
    Procesa un archivo NDJSON para contar las menciones de usuarios en los tweets,
    optimizando el tiempo de ejecución.

    Args:
        file_path (str): Ruta del archivo NDJSON.

    Returns:
        List[Tuple[str, int]]: Lista de los 10 usuarios más mencionados con sus frecuencias.
    """
    # Compilar la expresión regular para mejorar el rendimiento
    username_pattern = re.compile(r'@(\w+)')

    # Inicializar el contador de nombres de usuario
    username_counter = Counter()

    try:
        # Abrir el archivo en modo lectura
        with open(file_path, 'r') as file:
            # Usar una expresión generadora para reducir accesos a estructuras
            tweets_content = (
                json.loads(line).get('content', '') for line in file
            )

            # Actualizar el contador en un solo paso para optimizar la velocidad
            for content in tweets_content:
                try:
                    # Encontrar y contar los nombres de usuario en cada contenido
                    mentions = username_pattern.findall(content)
                    if mentions:
                        username_counter.update(mentions)
                except Exception as e:
                    # Registrar cualquier error inesperado
                    logger.warning(f"Error al procesar menciones: {e}")

    except (FileNotFoundError, IOError) as e:
        # Registrar error si no se puede abrir el archivo
        logger.error(f"Error al abrir el archivo: {e}")
        raise
    except json.JSONDecodeError as e:
        # Registrar error si una línea no se puede decodificar
        logger.error(f"Error al decodificar JSON: {e}")
        raise

    # Obtener los 10 usuarios más mencionados
    top_users = username_counter.most_common(10)

    # Registrar los resultados obtenidos
    logger.info(f"Top 10 usuarios mencionados: {top_users}")

    return top_users


In [27]:
%time q3_time(file_path)

CPU times: user 6.66 s, sys: 253 ms, total: 6.91 s
Wall time: 8.37 s


[('narendramodi', 2261),
 ('Kisanektamorcha', 1836),
 ('RakeshTikaitBKU', 1639),
 ('PMOIndia', 1422),
 ('RahulGandhi', 1125),
 ('GretaThunberg', 1046),
 ('RaviSinghKA', 1015),
 ('rihanna', 972),
 ('UNHumanRights', 962),
 ('meenaharris', 925)]

In [28]:
%memit q3_memory(file_path)

peak memory: 265.61 MiB, increment: 0.77 MiB
